# Spring Mass System - All Combos for C


Set up the synamics of a spring mass system. also generate all possible combinations of rows of the C matrix.

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter
from scipy.linalg import expm
from scipy.signal import cont2discrete
import itertools

# Spring-mass-damper parameters
m = 1.0  # Mass (kg)
c = 0.5  # Damping coefficient (Ns/m)
k = 2.0  # Spring constant (N/m)

# Continuous-time state-space matrices
A_c = np.array([[0, 1],
                [-k/m, -c/m]])
B_c = np.array([[0],
                [1/m]])

C_true = np.eye(2)

# Start off with the full C matrix
Cbig = np.array([
    [1, 0],
    [0, 1],
    [1, 1]
])
# Now, compute all unique combinations of C rows
unique_combos = []

# Get all unique combinations of rows
num_rows = len(Cbig)
for r in range(1, num_rows + 1):  # r determines subset size
    for combo in itertools.combinations(Cbig, r):
        #print(np.array(combo))  # Each combo is a tuple of rows
        unique_combos.append(combo)

C = []

for combo in unique_combos:
    padded_combo = np.zeros((3, Cbig.shape[1]), dtype=int)
    for i, row in enumerate(combo):
        padded_combo[i] = row
    C.append(padded_combo)

# Process and measurement noise
Q = 1e-4*np.eye(2)  # Process noise covariance
R = 0.05*np.eye(2) # Measurement noise covariance

# Discrete time system for kalman filter
dt = 0.1  # Time step
A_d = expm(A_c * dt)
B_d = np.linalg.solve(A_c, (A_d - np.eye(2))) @ B_c

# Initial conditions
x0 = np.array([1.0, 0.0])  # Initial displacement and velocity

Now, simulate the true system. 

In [49]:
# Simulate the true system
np.random.seed(42)
n_steps = 100 
true_states = []
measurements = []
for _ in range(n_steps):
    # Simulate true dynamics
    x_true = A_d @ x0 + np.random.multivariate_normal([0, 0], Q).T
    true_states.append(x_true)

    # Simulate noisy measurements
    z = C_true @ x_true + np.random.multivariate_normal([0, 0], R).T
    measurements.append(z)

    # Update for the next time step
    x0 = x_true

true_states = np.array(true_states)
measurements = np.array(measurements)

Then, use the kalman filter to generate state estimates for every combination of C. Make sure to use the 3 sigma range to set up the uncertainty range. 

In [50]:
nums = np.zeros(shape=(n_steps, 4))
data = []

for i in range(len(C)):
    system = (A_c, B_c, np.eye(2), 0)
    A_d, B_d, _, _, _ = cont2discrete(system, dt)
    Q = np.eye(2) * 1e-4
    R = np.eye(C[i].shape[0]) * 0.05

    # Kalman Filter setup
    kf = KalmanFilter(dim_x=2, dim_z=2)
    kf.F = A_d  # Discretized A matrix
    kf.H = C[i]  # Measurement matrix C
    kf.Q = Q  # Process noise covariance Q
    kf.R = R  # Measurement noise covariance R
    kf.P = np.eye(2) * 500  # Initial state covariance P
    kf.x = x0  # Initial state estimate

    data_at_this_C = pd.DataFrame(nums, columns=['position', 'velocity', 'position uncertainty', 'velocity uncertainty'])
    
    for j in range(len(measurements)): # use kalman filter to estimate the state and add it to my dataframe
        kf.predict()
        kf.update(measurements[j])
        data_at_this_C.iloc[j, 0]= kf.x[0]
        data_at_this_C.iloc[j, 1] = kf.x[1]
        three_sigma = 3 * np.sqrt(np.diag(kf.P)) # make sure to multiply by 3 for 3 sigma!!
        data_at_this_C.iloc[j, 2]= three_sigma[0]
        data_at_this_C.iloc[j, 3] = three_sigma[1]
    
    data.append(data_at_this_C.copy())

ValueError: operands could not be broadcast together with shapes (2,) (3,) 